# Varying Bin Size of Initial Digitization

In [1]:
#import all modules 
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier 
import pandas as pd
import numpy as np
import os
from numpy import asarray, save, load
import matplotlib.pyplot as plt

In [18]:
#insert the desired path containing combined data files
path = os.path.expanduser(r"~/Documents/python/identification_modified_proteins/")

#changes current working directory to the path given 
directory = os.chdir(path)

## Bin Size of 0.1 Units

In [16]:
#load data 
load_data = np.load('train_0.1_data.npy', allow_pickle=True)
df = pd.DataFrame(load_data)

df

,0,1
0,0.000000,0.0
1,0.000000,0.0
2,30.333333,0.0
3,32.999999,0.0
4,0.000000,0.0
...,...,...
1008,0.000000,0.0
1009,0.000000,0.0
1010,0.000000,0.0
1011,0.000000,0.0


In [9]:
#load data 
load_data = np.load('GYPK#AK 801.npy', allow_pickle=True)
df = pd.DataFrame(load_data)

df

,0
0,0.000000
1,0.000000
2,32.666668
3,0.000000
4,28.000000
...,...
101295,0.000000
101296,0.000000
101297,0.000000
101298,0.000000


In [13]:
#load data 
load_data = np.load('train_mod_np.npy', allow_pickle=True)
df = pd.DataFrame(load_data)

df

,0,1,2,3,4,5,6,7,8,9,...,101290,101291,101292,101293,101294,101295,101296,101297,101298,101299
0,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
#load data 
load_data = np.load('mod_train_data.npy', allow_pickle=True)
df = pd.DataFrame(load_data)

df

,0,1,2,3,4,5,6,7,8,9,...,101291,101292,101293,101294,101295,101296,101297,101298,101299,101300
0,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0.0,0.0,32.666668,0.0,28.0,86.666666,0.0,0.0,76.666669,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
